
# Face Recognition under Varying Visual Conditions 🧠📸
**Author:** Third Year ML Enthusiast  
**Task:** Robust Face Identity Prediction under diverse environmental conditions (fog, blur, glare, etc.) using MobileNetV2 and Test-Time Augmentation (TTA).  
**Libraries:** TensorFlow, Keras, OpenCV, Seaborn, sklearn

---

### 📝 Workflow Summary:
1. Load dataset using ImageDataGenerator with real-time augmentation.
2. Build and train a MobileNetV2-based model.
3. Fine-tune the model with last few layers unfreezed.
4. Evaluate performance globally and under specific visual distortions.
5. Predict with TTA + CLAHE enhancement.


In [ ]:

!pip install -q scikit-learn seaborn


In [ ]:

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from PIL import Image, ImageEnhance
import cv2


In [ ]:
#for mounting
# from google.colab import drive

# drive.mount('/content/drive')

#defining the paths of train and test folder
# zip_path = "/content/drive/MyDrive/Colab Notebooks/Comys_Hackathon5.zip"
# extract_path = "/content/drive/MyDrive/Comys_Hackathon5"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# Path to training data stored on Google Drive
base_dir = '/content/drive/MyDrive/Comys_Hackathon5/Comys_Hackathon5/Task_B/train'

# Image and training configurations
IMAGE_SIZE = (160, 160)
BATCH_SIZE = 32
EPOCHS = 5


In [ ]:

# Training & validation data generators with augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.15
)

# Load training set
train_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation set
val_gen = train_datagen.flow_from_directory(
    base_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes


In [ ]:

# Load MobileNetV2 (pretrained on ImageNet) without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_model.trainable = False  # Freeze base model

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compile model
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:

# Callbacks for saving best model and early stopping
checkpoint = ModelCheckpoint("initial_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Initial training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, earlystop]
)


In [ ]:

# Unfreeze last 30 layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune
fine_tune_history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=[earlystop]
)


In [ ]:

# Load trained model and prepare mapping
model = load_model("initial_model.h5")
idx_to_class = {v: k for k, v in train_gen.class_indices.items()}

# Enhance image using CLAHE (improves visibility under low-light, fog etc.)
def enhance_image_clahe(img):
    img = np.array(img)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0)
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

# Predict with TTA + CLAHE
def predict_face_identity(image_path, confidence_threshold=0.5):
    img = Image.open(image_path).convert("RGB")
    img = Image.fromarray(enhance_image_clahe(img)).resize((160, 160))

    # Variants: original, flipped, brightness adjusted
    variants = [
        img,
        img.transpose(Image.FLIP_LEFT_RIGHT),
        ImageEnhance.Brightness(img).enhance(0.7),
        ImageEnhance.Brightness(img).enhance(1.3)
    ]

    predictions = []
    for var in variants:
        arr = img_to_array(var)
        arr = preprocess_input(arr)
        arr = np.expand_dims(arr, axis=0)
        pred = model.predict(arr)
        predictions.append(pred)

    avg_pred = np.mean(predictions, axis=0)
    pred_index = np.argmax(avg_pred)
    confidence = avg_pred[0][pred_index]
    predicted_class = idx_to_class[pred_index]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class}\nConfidence: {confidence:.2f}")
    plt.show()

    if confidence < confidence_threshold:
        print("Low confidence prediction — may be inaccurate.")

    return predicted_class, confidence


In [ ]:

val_gen.reset()
preds = model.predict(val_gen, verbose=1)
y_true = val_gen.classes
y_pred = np.argmax(preds, axis=1)

# Metrics
top1_acc = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')

print(f"Top-1 Accuracy: {top1_acc:.4f}")
print(f"Macro F1 Score: {f1_macro:.4f}")
print("\nClassification Report:\n", classification_report(y_true, y_pred))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:

filenames = val_gen.filenames
idx_to_class = {v: k for k, v in val_gen.class_indices.items()}

def get_condition(fname):
    fname = fname.lower()
    for cond in ['blur', 'fog', 'rain', 'low', 'over', 'sun', 'glare', 'light']:
        if cond in fname:
            return cond
    return 'normal'

conditions = [get_condition(f) for f in filenames]
cond_perf = {}

# Group results by condition type
for i, cond in enumerate(conditions):
    if cond not in cond_perf:
        cond_perf[cond] = {'y_true': [], 'y_pred': []}
    cond_perf[cond]['y_true'].append(y_true[i])
    cond_perf[cond]['y_pred'].append(y_pred[i])

print("\nPerformance by Visual Condition:")
for cond, val in cond_perf.items():
    acc = accuracy_score(val['y_true'], val['y_pred'])
    f1 = f1_score(val['y_true'], val['y_pred'], average='macro')
    print(f"{cond.capitalize():<12} | Accuracy: {acc:.4f} | Macro F1: {f1:.4f}")


In [ ]:

# Visualize training + fine-tuning progress
def plot_training(history, fine_tune_history):
    acc = history.history['accuracy'] + fine_tune_history.history['accuracy']
    val_acc = history.history['val_accuracy'] + fine_tune_history.history['val_accuracy']
    loss = history.history['loss'] + fine_tune_history.history['loss']
    val_loss = history.history['val_loss'] + fine_tune_history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call plot function
plot_training(history, fine_tune_history)


In [ ]:

# Predict on a sample image
image_path = '/content/drive/MyDrive/Comys_Hackathon5/Comys_Hackathon5/Task_B/train/001_frontal/blur_face1.jpg'

label, confidence = predict_face_identity(image_path)
print(f"Predicted Identity: {label} | Confidence: {confidence:.2f}")
